# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Problema 2
#### Aluno: Eduardo Cesar Cauduro Coelho
#### Matrícula: 202310175
#### Aluno: Felipe Geraldo de Oliveira
#### Matrícula: 202310174

## Instalação da biblioteca Gurobipy





In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 55.0 MB/s eta 0:00:00


## Declaração dos parâmetros de capacidades das arestas do grafo:



In [2]:
vertices = ["A", "B", "Sul", "Norte"]

arestas = {
    "A": {"Norte": 300, "Sul": 500},
    "Sul": {"Norte": 300, "B": 300},
    "Norte": {"B": 400},
    "B": {}
}


## Criação do modelo:

In [3]:
from gurobipy import Model, GRB, quicksum

modelo = Model("transporte")

Restricted license - for non-production use only - expires 2026-11-23


## Declaração das variáveis de decisão que indicam o fluxo de barris de óleo entre um vértice e outro, caso haja a ligação.

In [4]:
variaveis = {}

for origem in arestas:
    for destino in arestas[origem]:
        variaveis[(origem, destino)] = modelo.addVar(vtype=GRB.CONTINUOUS, name=f"{origem}_{destino}", lb=0)

## Função objetivo: maximiza a quantidade de barris de óleo que chegam em B.

#### Alternativamente, poderíamos maximizar a quantidade de barris de óleo que saem de A, pois como não há estoque e temos somente uma origem e somente um destino, tudo que sai de A, chega efetivamente em B, e tudo que chega em B, é porque saiu de A.

In [5]:
somatorio = quicksum(variaveis[(origem, destino)]
                        for origem in arestas
                        for destino in arestas[origem]
                        if destino == "B")

modelo.setObjective(somatorio, GRB.MAXIMIZE)

## Restrições

* Como não há armazenamento nas estações intermediárias, tudo que chega nestas estações deve sair. Logo o fluxo de entrada é igual ao fluxo de saída:

In [6]:
for vertice in vertices:

  if(vertice == "A" or vertice == "B"):
    continue

  saida = quicksum(variaveis[(origem, destino)]
                        for origem in arestas
                        for destino in arestas[origem]
                        if origem == vertice)

  entrada = quicksum(variaveis[(origem, destino)]
                      for origem in arestas
                      for destino in arestas[origem]
                      if destino == vertice)

  modelo.addConstr(entrada == saida, name=f'conservacao_fluxo_{vertice}')

* O fluxo entre um par de vértices não deve ultrapassar a capacidade a aresta que liga os dois vértices:

In [7]:
for origem in arestas:
  for destino in arestas[origem]:
    modelo.addConstr(variaveis[(origem, destino)] <= arestas[origem][destino], f'capacidade_aresta_{origem}_{destino}')

## Portanto, temos o seguinte modelo:

\begin{equation}
\max f(x_{ij}) = ∑\limits_{i\mid(i,B)\in E}x_{iB}
\end{equation}

sujeito a:

\begin{alignat}{2}
∑\limits_{i\mid(i,j)\in E}x_{ij} = ∑\limits_{i\mid(j,i)\in E}x_{ji} && \qquad ∀j \in V' \\
x_{ij} \le C_{ij} && \qquad ∀(i,j) \in E \\
x_{ij} \ge 0 && \qquad
\end{alignat}

* Sendo:
  * G(V,E) o grafo com um conjunto de vértices V e um conjunto de arestas E
  * $x_{ij}$ as variáveis que representam o fluxo das arestas (i,j) contidas em A
  * $C_{ij}$ a capacidade da aresta (i,j)
  * V' representa V sem os vértices iniciais e finais, isto é V - \{A,B\}

# 1) Otimização do modelo:

In [8]:
modelo.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 5 columns and 11 nonzeros
Model fingerprint: 0x16ed832f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 5e+02]
Presolve removed 5 rows and 1 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+02   1.250000e+01   0.000000e+00      0s
       1    7.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.000000000e+02


## Impressão das soluções do modelo:

In [9]:
if(modelo.status == GRB.OPTIMAL):
  for origem in arestas:
    for destino in arestas[origem]:
        print(f"Barris de óleo transportados na rota {origem}->{destino}: {variaveis[(origem, destino)].x}")
  print(f"Valor ótimo: {modelo.objVal}")

Barris de óleo transportados na rota A->Norte: 300.0
Barris de óleo transportados na rota A->Sul: 400.0
Barris de óleo transportados na rota Sul->Norte: 100.0
Barris de óleo transportados na rota Sul->B: 300.0
Barris de óleo transportados na rota Norte->B: 400.0
Valor ótimo: 700.0


# 2) Análise de sensibilidade:

In [10]:
print("\nIntervalos de Custos (Vetor de Custos):")
for variavel in modelo.getVars():
  print(f"Variável {variavel.VarName}:")
  print(f"  Lower bound do custo (SAObjLow): {variavel.SAObjLow}")
  print(f"  Upper bound do custo (SAObjUp): {variavel.SAObjUp}")

print("\nIntervalos de Recursos (Vetor de Recursos):")
for restricao in modelo.getConstrs():
  print(f"Restrição {restricao.ConstrName}:")
  print(f"  Lower bound do recurso (SARHSLow): {restricao.SARHSLow}")
  print(f"  Upper bound do recurso (SARHSUp): {restricao.SARHSUp}")
  print(f"  Variavel dual: {restricao.pi}")
  print(f"  Folga primal: {restricao.slack}")


Intervalos de Custos (Vetor de Custos):
Variável A_Norte:
  Lower bound do custo (SAObjLow): -0.0
  Upper bound do custo (SAObjUp): inf
Variável A_Sul:
  Lower bound do custo (SAObjLow): -1.0
  Upper bound do custo (SAObjUp): 0.0
Variável Sul_Norte:
  Lower bound do custo (SAObjLow): -1.0
  Upper bound do custo (SAObjUp): 0.0
Variável Sul_B:
  Lower bound do custo (SAObjLow): -0.0
  Upper bound do custo (SAObjUp): inf
Variável Norte_B:
  Lower bound do custo (SAObjLow): -0.0
  Upper bound do custo (SAObjUp): inf

Intervalos de Recursos (Vetor de Recursos):
Restrição conservacao_fluxo_Sul:
  Lower bound do recurso (SARHSLow): -400.0
  Upper bound do recurso (SARHSUp): 100.0
  Variavel dual: 0.0
  Folga primal: 0.0
Restrição conservacao_fluxo_Norte:
  Lower bound do recurso (SARHSLow): -100.0
  Upper bound do recurso (SARHSUp): 100.0
  Variavel dual: 0.0
  Folga primal: 0.0
Restrição capacidade_aresta_A_Norte:
  Lower bound do recurso (SARHSLow): 200.0
  Upper bound do recurso (SARHSUp)

##### Os valores de Lower Bound e Upper Bound dos custos apresentam valores pouco significativos, com limitantes inferiores assumindo valor 0 e -1 e superiores assumindo valores 0 e infinito. Isso faz sentido, já que alterar o custo de determinadas variáveis como o fluxo entre estações intermediárias não faz sentido segundo a lógica do problema, que é maximizar a quantidade de produtos que chegam em B. Considerar os produtos que saem de A quando já se considera os produtos que chegam em B também não faz sentido, já que tudo que sai de A, chega em B. Considerar custos diferentes de 1 para as variáveis que chegam em B também não faz sentido segundo a lógica do problema.

##### Os valores de Lower Bound e Upper Bound dos recursos relacionados às variáveis de consevação de fluxo também são pouco relevantes, já que a conservação de fluxo presume que o recurso é 0, pois, se não fosse, não haveria conservação de fluxo. Os valores das variáveis duais relacionadas à essas restrições é 0.

##### Já Os valores de Lower Bound, Upper Bound dos recursos relacionados às capacidades das arestas e as variáveis duais relacionadas a essas restrições apresentam valores interessantes de se analisar, pois a capacidade de uma aresta pode variar, dependendo de diversas condições do percurso, já que o grafo modela o transporte de barris de óleo.


##### Os valores de Upper Bound da aresta A->Sul e Sul->Norte possuem o valor infinito e as variáveis duais apresentam valor 0, indicando que independentemente de quanto aumente os recursos dessas variáveis, a solução básica não vai se alterar e tampouco haverá algum aumento na função objetivo. Isso se dá pois o fluxo é 400 e 100, enquanto suas capacidades são 500 e 300, para as arestas A->Sul e Sul->Norte, respectivamente, indicando que elas não são o gargalo do fluxo, e, portanto, aumentar suas capacidades não é eficiente.

##### A aresta A->Norte que, embora tenha sua capacidade total utilizada, aumentar sua capacidade não aumenta a função objetivo (variável dual = 0), pois há um gargalo posterior: a capacidade de chegada em B é inferior à capacidade de saída em A e aumentar a capacidade de saída em A não é efetivo se não aumentar a capacidade de chegada em B.

##### Já, por último, aumentar os valores dos recursos de Sul->B e Norte->B há uma melhoria da função objetivo, pois essas arestas estão completamente utilizadas, ou seja, seu fluxo é igual a sua capacidade, e, aumentando sua capacidade em 1 unidade, seu fluxo é aumentado em 1 unidade. Outro ponto a se destacar é que o Upper Bound da aresta Norte->B é 400, que é igual a sua capacidade, isso indica que qualquer aumento significa uma mudança na solução básica.